<a href="https://colab.research.google.com/github/bhavanadevarashetty888/Bhavana_INFO5731_Spring2021/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

In [2]:
import pandas as pd
import string
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
def fetch_data(driver, class_name_val, css_selector_val):
  result = list()
  values = driver.find_elements_by_class_name(class_name_val)
  for value in values:
    result.append(value.find_element_by_css_selector(css_selector_val).text)
  return result
  pass

In [5]:
import time
driver.get("https://www.imdb.com/title/tt7286456/reviews?sort=userRating&dir=desc&ratingFilter=0")
i=0
while(i<3):
  button = driver.find_element_by_xpath('//*[@id ="load-more-trigger"]').click()
  i = i+1
  time.sleep(2)

reviews = fetch_data(driver, "content", ".text.show-more__control")
user_names = fetch_data(driver, "display-name-date","a")

In [6]:

required_df = pd.concat([pd.DataFrame(user_names,columns=['User Names']),pd.DataFrame(reviews,columns=['Reviews'])],axis=1)
required_df

,User Names,Reviews
0,morino-59166,When I saw the magnificent Heath ledger a few ...
1,asassandroo,I can say that i'm really grateful that movies...
2,samanehdds,I'm not a fan of Batman movies but went to see...
3,guillevica,"What a master piece, it so so wondeful, all in..."
4,ahurworth,Has to be the best movie I've seen in years. I...
...,...,...
95,tusharshanker-96210,"This movie is truly a masterpiece. The acting,..."
96,a_z_love-28027,One of the best movies in my life .. story .. ...
97,manchukaman,"Long story short, nothing to say other than wh..."
98,hiranthas,"Being rejected by the society, fired from the ..."


In [7]:
required_df["Punctuations"]=required_df['Reviews'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
required_df["spl chars"] = required_df["Punctuations"].apply(lambda x:''.join([i for i in x if i.isalnum() or i == " "]))
required_df["numbers"] = required_df['spl chars'].apply(lambda x:''.join([i for i in x if i == " " or i.isalpha()]))

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
list_of_words = set(stopwords.words('english'))
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer()

nltk.download('wordnet')
import textblob

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
required_df["stop_words"]=required_df['numbers'].apply(lambda x:' '.join([i for i in x.split() if i not in list_of_words]))
required_df["lower_case"]=required_df['stop_words'].apply(lambda x: x.lower())
required_df["tokenisation"]=required_df["lower_case"].apply(lambda x : word_tokenize(str(x)))
required_df["stemming"]=required_df["tokenisation"].apply(lambda x: ' '.join([ps.stem(i) for i in x]))
required_df["lemmitization"]=required_df['tokenisation'].apply(lambda x: ' '.join([textblob.Word(i).lemmatize() for i in x]))

final_df = required_df[["User Names","Reviews","lemmitization"]]

final_df

,User Names,Reviews,lemmitization
0,morino-59166,When I saw the magnificent Heath ledger a few ...,when i saw magnificent heath ledger year go ex...
1,asassandroo,I can say that i'm really grateful that movies...,i say im really grateful movie like one havent...
2,samanehdds,I'm not a fan of Batman movies but went to see...,im fan batman movie went see teenage son it tr...
3,guillevica,"What a master piece, it so so wondeful, all in...",what master piece wondeful movie great constru...
4,ahurworth,Has to be the best movie I've seen in years. I...,ha best movie ive seen year it feel taxi drive...
...,...,...,...
95,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",this movie truly masterpiece the acting cenima...
96,a_z_love-28027,One of the best movies in my life .. story .. ...,one best movie life story acting director soun...
97,manchukaman,"Long story short, nothing to say other than wh...",long story short nothing say said any movie fa...
98,hiranthas,"Being rejected by the society, fired from the ...",being rejected society fired job lover happine...


In [11]:
import itertools
from itertools import chain
from nltk import ngrams
from collections import Counter

In [12]:
data_set = final_df["lemmitization"].values.tolist()
result=list()
required_string = str()

In [13]:
for data in data_set:
  result.append(word_tokenize(data))
  required_string=required_string+' '+data
def fetch_list_of_iterables(val):
  return list(itertools.chain.from_iterable(val))

result2 = [i for i in result if len(i)!= 0]
Counter(ngrams(fetch_list_of_iterables(result2),3))

Counter({('when', 'i', 'saw'): 1,
         ('i', 'saw', 'magnificent'): 1,
         ('saw', 'magnificent', 'heath'): 1,
         ('magnificent', 'heath', 'ledger'): 1,
         ('heath', 'ledger', 'year'): 1,
         ('ledger', 'year', 'go'): 1,
         ('year', 'go', 'excellent'): 1,
         ('go', 'excellent', 'role'): 1,
         ('excellent', 'role', 'joker'): 1,
         ('role', 'joker', 'i'): 1,
         ('joker', 'i', 'doubted'): 1,
         ('i', 'doubted', 'one'): 1,
         ('doubted', 'one', 'make'): 1,
         ('one', 'make', 'amazing'): 1,
         ('make', 'amazing', 'performance'): 1,
         ('amazing', 'performance', 'heath'): 1,
         ('performance', 'heath', 'year'): 1,
         ('heath', 'year', 'heath'): 1,
         ('year', 'heath', 'tragic'): 1,
         ('heath', 'tragic', 'death'): 1,
         ('tragic', 'death', 'break'): 1,
         ('death', 'break', 'heart'): 1,
         ('break', 'heart', 'joaquin'): 1,
         ('heart', 'joaquin', 'phoenix'): 1

In [14]:
#Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

from nltk import bigrams
from collections import Counter

bigrams_data = bigrams(fetch_list_of_iterables(result2))
bigrams_list = dict(Counter(bigrams_data))

res1=fetch_list_of_iterables(result2)

In [15]:
for i in bigrams_list:
  if(i!=None):
    print(i," :  ",bigrams_list.get(i)/res1.count(i[0]))

('when', 'i')  :   0.5
('i', 'saw')  :   0.045871559633027525
('saw', 'magnificent')  :   0.14285714285714285
('magnificent', 'heath')  :   0.5
('heath', 'ledger')  :   0.5714285714285714
('ledger', 'year')  :   0.16666666666666666
('year', 'go')  :   0.058823529411764705
('go', 'excellent')  :   0.05263157894736842
('excellent', 'role')  :   0.2
('role', 'joker')  :   0.125
('joker', 'i')  :   0.07462686567164178
('i', 'doubted')  :   0.009174311926605505
('doubted', 'one')  :   1.0
('one', 'make')  :   0.023255813953488372
('make', 'amazing')  :   0.04
('amazing', 'performance')  :   0.038461538461538464
('performance', 'heath')  :   0.037037037037037035
('heath', 'year')  :   0.14285714285714285
('year', 'heath')  :   0.058823529411764705
('heath', 'tragic')  :   0.14285714285714285
('tragic', 'death')  :   0.5
('death', 'break')  :   1.0
('break', 'heart')  :   1.0
('heart', 'joaquin')  :   0.16666666666666666
('joaquin', 'phoenix')  :   0.6944444444444444
('phoenix', 'bring')  :  

In [16]:
#Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

reviews = required_df["lower_case"].values.tolist()
total_reviews_list = " | ".join(reviews)

In [22]:
!pip install textacy
import textacy

In [23]:

import textacy
frequency = list()
nouns =list()
tokens_list_1 = list()

In [24]:
for i in reviews:
  a_1=list()
  # print(i)
  review_doc = textacy.make_spacy_doc(i,lang='en_core_web_sm')
  #print(review_doc)
  for j in review_doc.noun_chunks:
    if(str(j)!="None" and str(j)!="i"):
      a_1.append(str(j))

tokens_list_1.append(a_1)

print(tokens_list_1)

[['m big fan batman series', 'especially joker', 'movie', 'it', 'art', 'far best movie', 'chance', 'masterpieces', 'one dc']]


In [25]:
#compute nouns frequency
for i in tokens_list_1:
  for j in i:
    b_1=list()
    for review in reviews:
      if(str(review)!="None" and str(review)!="i"):
      # print(review.count(j)/total_reviews_list.count(j))
        b_1.append(review.count(j)/total_reviews_list.count(j))
    nouns.append(b_1)

list_of_nouns = fetch_list_of_iterables(tokens_list_1)
result_df=pd.DataFrame(nouns)

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [26]:
# Write your code here
res1 = [review.split(" ") for review in reviews]
final_set = fetch_list_of_iterables(res1)

In [27]:
words=list()
for i in res1:
  dict_1 = dict.fromkeys(final_set,0)
  for j in i:
    dict_1[j]+= 1
  words.append(dict_1)
res = pd.DataFrame(words)
required_words = res.columns
print("Frequency of each word in list")
result1 = res.T
result1.head()

Frequency of each word in list


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
when,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
i,6,5,4,1,1,0,0,2,1,2,1,0,0,0,1,0,0,0,4,0,3,4,1,0,0,0,3,1,0,0,1,0,0,0,0,5,1,0,0,0,...,0,0,0,2,0,0,5,1,0,1,2,0,0,0,1,0,0,0,0,0,6,0,0,0,0,0,4,1,0,5,1,0,0,0,1,0,0,1,0,3
saw,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
magnificent,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heath,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
import numpy as np
import math
def computeTF(word,total): 
  value = len(total)
  return total.count(word)/value
  pass

def computeIDF(word,total):
  res = total.count(word)
  if(res!=0):
    return math.log(len(total)/res, 10)
  else:
    return 0
  pass

result_tf_idf = pd.DataFrame(required_words,columns=["words"])
counter=1
for value in res1:
  result_tf_idf["tf"] = result_tf_idf["words"].apply(lambda x : computeTF(x,value))
  result_tf_idf["idf"] = result_tf_idf["words"].apply(lambda x : computeIDF(x,value))
  result_tf_idf["review "+str(counter)] = result_tf_idf["tf"]*result_tf_idf["idf"]
  counter=counter+1

result_tf_idf.drop(["tf","idf"],axis=1).head(10)

,words,review 1,review 2,review 3,review 4,review 5,review 6,review 7,review 8,review 9,review 10,review 11,review 12,review 13,review 14,review 15,review 16,review 17,review 18,review 19,review 20,review 21,review 22,review 23,review 24,review 25,review 26,review 27,review 28,review 29,review 30,review 31,review 32,review 33,review 34,review 35,review 36,review 37,review 38,review 39,...,review 61,review 62,review 63,review 64,review 65,review 66,review 67,review 68,review 69,review 70,review 71,review 72,review 73,review 74,review 75,review 76,review 77,review 78,review 79,review 80,review 81,review 82,review 83,review 84,review 85,review 86,review 87,review 88,review 89,review 90,review 91,review 92,review 93,review 94,review 95,review 96,review 97,review 98,review 99,review 100
0,when,0.027255,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.030014,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,i,0.093844,0.08260,0.073076,0.065051,0.040051,0.0,0.0,0.080094,0.046016,0.030805,0.047036,0.0,0.0,0.000000,0.051684,0.0,0.0,0.0,0.066156,0.0,0.093032,0.082786,0.048108,0.000000,0.0,0.0,0.065783,0.078406,0.0,0.000000,0.009173,0.0,0.0,0.000000,0.0,0.060279,0.069737,0.0,0.000000,...,0.0,0.0,0.0,0.055918,0.0,0.0,0.096727,0.045043,0.0,0.094672,0.094672,0.000000,0.0,0.0,0.072379,0.0,0.0,0.0,0.0,0.0,0.078960,0.0,0.0,0.0,0.0,0.0,0.067893,0.041573,0.000000,0.089932,0.055918,0.0,0.0,0.0,0.120728,0.0,0.000000,0.027255,0.0,0.089932
2,saw,0.027255,0.00000,0.027255,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.024258,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.030014,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.032534,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,magnificent,0.027255,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,heath,0.060401,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.046016,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.021166,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.072379,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
5,ledger,0.027255,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.072379,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
6,years,0.06040

In [29]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
wordnet=WordNetLemmatizer()

query = "An Outstanding movie with a haunting performance and best character development"

import re

query = query.lower()
query = re.sub("[0-9]","",query)
x = ""
for i in query:
  if (i.isalnum() or i.isalpha() or i == " "):
    x = x+i
query = x

print(query)

an outstanding movie with a haunting performance and best character development


In [30]:
list_of_sentences= word_tokenize(query) 
stop_words= stopwords.words('english') 
set_y = [value for value in list_of_sentences if not value in stop_words]
nltk.download('wordnet')

lemmetized_Sentences=' '.join([wordnet.lemmatize(value) for value in set_y])
lemmetized_Sentences=set(word_tokenize(lemmetized_Sentences))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
def fetch_x_y(x,y,ip1,res,lemmetized_Sentences):
  for i in ip1:
   if i in res:
     x.append(1)
   else:
     x.append(0)
   if i in lemmetized_Sentences:
     y.append(1)
   else:
     y.append(0)
  return x,y
  pass

In [32]:
def fetch_cos(cos_value,x,y):
  #apply formula
  denominator = float(pow((sum(x)*sum(y)),0.5))
  if(denominator!=0):
    cos_value = cos_value/denominator
  else:
    cos_value=0
  return cos_value
  pass

In [33]:
similarity = list()

for sentence in reviews:
  res = set(word_tokenize(sentence))
  ip1 = res.union(lemmetized_Sentences)
  x=list()
  y=list()
  x,y = fetch_x_y(x,y,ip1,res,lemmetized_Sentences)
  cos_value = 0
  size = len(ip1)
  for i in range(size):
    cos_value = cos_value + (x[i]*y[i])
  cos_value = fetch_cos(cos_value,x,y)
  similarity.append(cos_value)

values = pd.DataFrame(reviews,columns=["reviews"])
values["similarity"] = similarity
values

,reviews,similarity
0,when i saw magnificent heath ledger years go e...,0.106904
1,i say im really grateful movies like one haven...,0.050508
2,im fan batman movies went see teenage son its ...,0.100125
3,what master piece wondeful movie great constru...,0.086711
4,has best movie ive seen years it feel taxi dri...,0.124274
...,...,...
95,this movie truly masterpiece the acting cenima...,0.104828
96,one best movies life story acting director sou...,0.173422
97,long story short nothing say said any movie fa...,0.050508
98,being rejected society fired job lover happine...,0.106904


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [34]:
# The GitHub link of your final csv file

required_df[["User Names","Reviews","lower_case"]].to_csv("sentimentAnal.csv")

In [ ]:
Github link

https://github.com/bhavanadevarashetty888/Bhavana_INFO5731_Spring2021/blob/main/sentimentAnal.csv